# Email Scraper

import deps

In [ ]:
import grequests
from bs4 import BeautifulSoup
import re
import json

read list of URLs to scrape from `in.txt`

In [ ]:
input_file = 'in.json'

with open('./in/' + input_file) as f:
    urls = json.load(f)

urls

scrape emails from anchor tags with mailto links

In [ ]:
def extract_emails(response):
  
  # Parse the HTML content
  soup = BeautifulSoup(response.content, 'html.parser')

  # Find all 'mailto:' links
  mailto_links = soup.find_all('a', href=re.compile(r'^mailto:'))

  # Extract email addresses from 'mailto:' links
  emails = set()
  for link in mailto_links:
    email = link['href'].replace('mailto:', '')
    query_string_start = email.find('?')
    if query_string_start >= 0:
      email = email[0:query_string_start]
    emails.add(email)

  return emails

In [ ]:
requests = (grequests.get(u) for u in urls)

email_addresses = set()
for response in grequests.imap(requests, size=10):
  email_addresses.update(extract_emails(response))

In [ ]:
print(f"Total email addresses: {len(email_addresses):0}")
email_addresses

write scraped email address to file

In [ ]:
output_file = 'out.txt'

with open('./out/' + output_file, 'w+') as f:
    f.writelines(", ".join(email_addresses))